In [17]:
from elasticsearch import Elasticsearch, helpers
from dotenv import load_dotenv
from pprint import pp
import os
load_dotenv("../elastic-start-local/.env")
ES_LOCAL_API_KEY = os.environ.get("ES_LOCAL_API_KEY")
ES_LOCAL_URL = os.environ.get("ES_LOCAL_URL")

def ppr(resp):
    pp(resp.raw)

In [5]:
client = Elasticsearch([ES_LOCAL_URL], api_key=ES_LOCAL_API_KEY)

In [7]:
resp = client.indices.create(
    index="cooking_blog",
)

ppr(resp)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'cooking_blog'}


In [8]:
# standard analyzer is used by default for text fields
# text are both text and keyword to allow full-text search and exact matching/filtering on the same field. Done automatically with dynamic mapping
# ignore_above, prevents indexing values longer than 256, by default it's already 256

resp = client.indices.put_mapping(
    index="cooking_blog",
    properties={
        "title": {
            "type": "text",
            "analyzer": "standard",
            "fields": {
                "keyword": {
                    "type": "keyword",
                    "ignore_above": 256
                }
            }
        },
        "description": {
            "type": "text",
            "fields": {
                "keyword": {
                    "type": "keyword"
                }
            }
        },
        "author": {
            "type": "text",
            "fields": {
                "keyword": {
                    "type": "keyword"
                }
            }
        },
        "date": {
            "type": "date",
            "format": "yyyy-MM-dd"
        },
        "category": {
            "type": "text",
            "fields": {
                "keyword": {
                    "type": "keyword"
                }
            }
        },
        "tags": {
            "type": "text",
            "fields": {
                "keyword": {
                    "type": "keyword"
                }
            }
        },
        "rating": {
            "type": "float"
        }
    },
)

ppr(resp)

ObjectApiResponse({'acknowledged': True})


In [9]:
resp = client.bulk(
    index="cooking_blog",
    refresh="wait_for",
    operations=[
        {
            "index": {
                "_id": "1"
            }
        },
        {
            "title": "Perfect Pancakes: A Fluffy Breakfast Delight",
            "description": "Learn the secrets to making the fluffiest pancakes, so amazing you won't believe your tastebuds. This recipe uses buttermilk and a special folding technique to create light, airy pancakes that are perfect for lazy Sunday mornings.",
            "author": "Maria Rodriguez",
            "date": "2023-05-01",
            "category": "Breakfast",
            "tags": [
                "pancakes",
                "breakfast",
                "easy recipes"
            ],
            "rating": 4.8
        },
        {
            "index": {
                "_id": "2"
            }
        },
        {
            "title": "Spicy Thai Green Curry: A Vegetarian Adventure",
            "description": "Dive into the flavors of Thailand with this vibrant green curry. Packed with vegetables and aromatic herbs, this dish is both healthy and satisfying. Don't worry about the heat - you can easily adjust the spice level to your liking.",
            "author": "Liam Chen",
            "date": "2023-05-05",
            "category": "Main Course",
            "tags": [
                "thai",
                "vegetarian",
                "curry",
                "spicy"
            ],
            "rating": 4.6
        },
        {
            "index": {
                "_id": "3"
            }
        },
        {
            "title": "Classic Beef Stroganoff: A Creamy Comfort Food",
            "description": "Indulge in this rich and creamy beef stroganoff. Tender strips of beef in a savory mushroom sauce, served over a bed of egg noodles. It's the ultimate comfort food for chilly evenings.",
            "author": "Emma Watson",
            "date": "2023-05-10",
            "category": "Main Course",
            "tags": [
                "beef",
                "pasta",
                "comfort food"
            ],
            "rating": 4.7
        },
        {
            "index": {
                "_id": "4"
            }
        },
        {
            "title": "Vegan Chocolate Avocado Mousse",
            "description": "Discover the magic of avocado in this rich, vegan chocolate mousse. Creamy, indulgent, and secretly healthy, it's the perfect guilt-free dessert for chocolate lovers.",
            "author": "Alex Green",
            "date": "2023-05-15",
            "category": "Dessert",
            "tags": [
                "vegan",
                "chocolate",
                "avocado",
                "healthy dessert"
            ],
            "rating": 4.5
        },
        {
            "index": {
                "_id": "5"
            }
        },
        {
            "title": "Crispy Oven-Fried Chicken",
            "description": "Get that perfect crunch without the deep fryer! This oven-fried chicken recipe delivers crispy, juicy results every time. A healthier take on the classic comfort food.",
            "author": "Maria Rodriguez",
            "date": "2023-05-20",
            "category": "Main Course",
            "tags": [
                "chicken",
                "oven-fried",
                "healthy"
            ],
            "rating": 4.9
        }
    ],
)
ppr(resp)

ObjectApiResponse({'errors': False, 'took': 800, 'items': [{'index': {'_index': 'cooking_blog', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'cooking_blog', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'cooking_blog', '_id': '3', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'cooking_blog', '_id': '4', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'cooking_blog', '_id': '5', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 1, 'st

In [11]:
# FTS, by default it uses OR for the tokens in the query

resp = client.search(
    index="cooking_blog",
    query={
        "match": {
            "description": {
                "query": "fluffy pancakes"
            }
        }
    },
)
ppr(resp)

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
          'max_score': 1.8378843,
          'hits': [{'_index': 'cooking_blog',
                    '_id': '1',
                    '_score': 1.8378843,
                    '_source': {'title': 'Perfect Pancakes: A Fluffy Breakfast '
                                         'Delight',
                                'description': 'Learn the secrets to making '
                                               'the fluffiest pancakes, so '
                                               "amazing you won't believe your "
                                               'tastebuds. This recipe uses '
                                               'buttermilk and a special '
                                               'folding technique to create '
                                               'light, airy pancakes that are '
             

In [18]:
# FTS, but with and for the tokens

resp = client.search(
    index="cooking_blog",
    query={
        "match": {
            "description": {
                "query": "fluffy pancakes",
                "operator": "and"
            }
        }
    },
)
ppr(resp)

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
          'max_score': None,
          'hits': []}}


In [20]:
resp = client.search(
    index="cooking_blog",
    query={
        "match": {
            "title": {
                "query": "fluffy pancakes breakfast",
                "minimum_should_match": 2
            }
        }
    },
)
ppr(resp)

{'took': 13,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
          'max_score': 4.0408072,
          'hits': [{'_index': 'cooking_blog',
                    '_id': '1',
                    '_score': 4.0408072,
                    '_source': {'title': 'Perfect Pancakes: A Fluffy Breakfast '
                                         'Delight',
                                'description': 'Learn the secrets to making '
                                               'the fluffiest pancakes, so '
                                               "amazing you won't believe your "
                                               'tastebuds. This recipe uses '
                                               'buttermilk and a special '
                                               'folding technique to create '
                                               'light, airy pancakes that are '
            

In [22]:
resp = client.search(
    index="cooking_blog",
    query={
        "multi_match": {
            "query": "vegetarian curry",
            "fields": [
                "title",
                "description",
                "tags"
            ]
        }
    },
)
ppr(resp)

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
          'max_score': 2.8276732,
          'hits': [{'_index': 'cooking_blog',
                    '_id': '2',
                    '_score': 2.8276732,
                    '_source': {'title': 'Spicy Thai Green Curry: A Vegetarian '
                                         'Adventure',
                                'description': 'Dive into the flavors of '
                                               'Thailand with this vibrant '
                                               'green curry. Packed with '
                                               'vegetables and aromatic herbs, '
                                               'this dish is both healthy and '
                                               "satisfying. Don't worry about "
                                               'the heat - you can easily '
              

In [24]:
resp = client.search(
    index="cooking_blog",
    query={
        "multi_match": {
            "query": "vegetarian curry",
            "fields": [
                "title^3",
                "description^2",
                "tags"
            ]
        }
    },
)
ppr(resp)

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
          'max_score': 7.546015,
          'hits': [{'_index': 'cooking_blog',
                    '_id': '2',
                    '_score': 7.546015,
                    '_source': {'title': 'Spicy Thai Green Curry: A Vegetarian '
                                         'Adventure',
                                'description': 'Dive into the flavors of '
                                               'Thailand with this vibrant '
                                               'green curry. Packed with '
                                               'vegetables and aromatic herbs, '
                                               'this dish is both healthy and '
                                               "satisfying. Don't worry about "
                                               'the heat - you can easily '
                

In [26]:
resp = client.search(
    index="cooking_blog",
    query={
        "bool": {
            "filter": [
                {
                    "term": {
                        "category.keyword": "Breakfast"
                    }
                }
            ]
        }
    },
)
ppr(resp)

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
          'max_score': 0.0,
          'hits': [{'_index': 'cooking_blog',
                    '_id': '1',
                    '_score': 0.0,
                    '_source': {'title': 'Perfect Pancakes: A Fluffy Breakfast '
                                         'Delight',
                                'description': 'Learn the secrets to making '
                                               'the fluffiest pancakes, so '
                                               "amazing you won't believe your "
                                               'tastebuds. This recipe uses '
                                               'buttermilk and a special '
                                               'folding technique to create '
                                               'light, airy pancakes that are '
                         

In [27]:
resp = client.search(
    index="cooking_blog",
    query={
        "range": {
            "date": {
                "gte": "2023-05-01",
                "lte": "2023-05-31"
            }
        }
    },
)
ppr(resp)

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5, 'relation': 'eq'},
          'max_score': 1.0,
          'hits': [{'_index': 'cooking_blog',
                    '_id': '1',
                    '_score': 1.0,
                    '_source': {'title': 'Perfect Pancakes: A Fluffy Breakfast '
                                         'Delight',
                                'description': 'Learn the secrets to making '
                                               'the fluffiest pancakes, so '
                                               "amazing you won't believe your "
                                               'tastebuds. This recipe uses '
                                               'buttermilk and a special '
                                               'folding technique to create '
                                               'light, airy pancakes that are '
                         

In [28]:
resp = client.search(
    index="cooking_blog",
    query={
        "term": {
            "author.keyword": "Maria Rodriguez"
        }
    },
)
ppr(resp)

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
          'max_score': 0.87546873,
          'hits': [{'_index': 'cooking_blog',
                    '_id': '1',
                    '_score': 0.87546873,
                    '_source': {'title': 'Perfect Pancakes: A Fluffy Breakfast '
                                         'Delight',
                                'description': 'Learn the secrets to making '
                                               'the fluffiest pancakes, so '
                                               "amazing you won't believe your "
                                               'tastebuds. This recipe uses '
                                               'buttermilk and a special '
                                               'folding technique to create '
                                               'light, airy pancakes that are '
           

In [29]:
resp = client.search(
    index="cooking_blog",
    query={
        "bool": {
            "must": [
                {
                    "term": {
                        "category.keyword": "Main Course"
                    }
                },
                {
                    "term": {
                        "tags": "vegetarian"
                    }
                },
                {
                    "range": {
                        "rating": {
                            "gte": 4.5
                        }
                    }
                }
            ],
            "should": [
                {
                    "multi_match": {
                        "query": "curry spicy",
                        "fields": [
                            "title^2",
                            "description"
                        ]
                    }
                },
                {
                    "range": {
                        "date": {
                            "gte": "now-1M/d"
                        }
                    }
                }
            ],
            "must_not": [
                {
                    "term": {
                        "category.keyword": "Dessert"
                    }
                }
            ]
        }
    },
)
ppr(resp)

{'took': 9,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
          'max_score': 7.9835095,
          'hits': [{'_index': 'cooking_blog',
                    '_id': '2',
                    '_score': 7.9835095,
                    '_source': {'title': 'Spicy Thai Green Curry: A Vegetarian '
                                         'Adventure',
                                'description': 'Dive into the flavors of '
                                               'Thailand with this vibrant '
                                               'green curry. Packed with '
                                               'vegetables and aromatic herbs, '
                                               'this dish is both healthy and '
                                               "satisfying. Don't worry about "
                                               'the heat - you can easily '
              